# CREACIÓN DE NUEVAS TABLAS

# Review:

Importamos librerias

In [2]:
import pandas as pd
import uuid #es para generar ids

Cargamos la data

In [3]:
review_google = pd.read_json("C:/Users/leand/Desktop/Proyecto Final/Proyecto-Final-HENRY-Big-Data/ETL/Data florida/reviews_Florida_Google.json")

In [4]:
reviews_fl = pd.read_json("C:/Users/leand/Desktop/Proyecto Final/Proyecto-Final-HENRY-Big-Data/ETL/Data florida/yelp_reviews_fl.json")

In [5]:
reviews_fl.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5,2,0,0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16


In [6]:
review_google.head(1)

,user_id,name,time,rating,text,gmap_id
0,1.111923e+20,Nellie Lao,1467403800000,5,Excellent Dominican/PuertoRican food. Her Arro...,0x88dae191ee505917:0x6ba3e25388d3fad4


Agregamos un id al df review_google

In [7]:
review_google['id_review_google'] = [str(uuid.uuid4()) for _ in range(len(review_google))]

Convertimos la columna time al formato correcto

In [8]:
review_google['time'] = pd.to_datetime(review_google['time'], unit='ms')
review_google['time'] = review_google['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

Transformamos las columnas de los df

In [9]:
# Renombrar las columnas del segundo DataFrame para que coincidan con las del primero
review_google.rename(columns={'time': 'date', 'id_review_google': 'review_id', 'rating': 'stars'}, inplace=True)

# Agregar una columna 'id_restaurante' con valores nulos para ambos DataFrames
reviews_fl['id_restaurante'] = reviews_fl['business_id']
review_google['id_restaurante'] = review_google['gmap_id']

# Agregar una columna 'id_usuario' para ambos DataFrames
reviews_fl['id_usuario'] = reviews_fl['user_id']
review_google['id_usuario'] = review_google['user_id']

# Agregar una columna 'id_plataforma' para ambos DataFrames
reviews_fl['id_plataforma'] = 'Yelp'
review_google['id_plataforma'] = 'Google'

Unimos los df

In [10]:
# Unir ambos DataFrames
df_combined = pd.concat([reviews_fl, review_google], ignore_index=True)

la columna identificador_plataforma tendra el id original de su respectiva plataforma

In [11]:
# Crear una nueva columna 'identificador_plataforma' según la plataforma
df_combined['identificador_plataforma'] = df_combined.apply(lambda row: row['review_id'] if pd.notnull(row['review_id']) else row['id_review_google'], axis=1)


Creamos un nuveo id a la columna review_id

In [12]:
df_combined['review_id'] = range(len(df_combined))

Elegimos las columnas que vamos a querer para el nuevo df

In [13]:
# Seleccionar las columnas requeridas
df_reviews = df_combined[['review_id', 'id_restaurante', 'id_usuario', 'id_plataforma', 'identificador_plataforma',
                          'stars', 'text', 'date', 'useful', 'funny', 'cool']]

Renombramos algunas columnas

In [14]:
# Renombrar las columnas del segundo DataFrame para que coincidan con las del primero
df_reviews.rename(columns={'stars': 'raiting', 'review_id': 'id_review', 'text': 'review'}, inplace=True)

C:\Users\leand\AppData\Local\Temp\ipykernel_4156\2036404310.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews.rename(columns={'stars': 'raiting', 'review_id': 'id_review', 'text': 'review'}, inplace=True)


In [15]:
# Mostrar el nuevo DataFrame
df_reviews.tail(1)

,id_review,id_restaurante,id_usuario,id_plataforma,identificador_plataforma,raiting,review,date,useful,funny,cool
1931712,1931712,0x88e77b72f6649745:0x79b4f1ce48d76510,113792330699999985664.0,Google,a92477e7-7912-4425-85d7-b45dffb7cf25,4,None,2019-04-02 15:12:00,NaN,NaN,NaN


Exportamos el df

In [16]:
'''
df_reviews.to_json('reviews.json', orient='records')
'''